# Part 1.6: Optimization Theory for Machine Learning

Every machine learning model learns by optimizing an objective function. Understanding optimization theory
is what separates practitioners who *use* ML from those who *understand* it. When your model fails to
converge, when training is painfully slow, or when your network mysteriously generalizes despite having
millions of parameters — optimization theory tells you why.

This notebook covers the theoretical foundations: convergence guarantees, the bias-variance tradeoff,
generalization theory, and regularization. These ideas form the bridge between the math foundations
of Part 1 and the practical deep learning of Parts 3+.

## Learning Objectives

- [ ] Analyze gradient descent convergence rates for convex and strongly convex functions
- [ ] Explain why convexity guarantees global optima and how condition number affects convergence
- [ ] Compare gradient descent vs. stochastic gradient descent and understand mini-batch tradeoffs
- [ ] Derive the bias-variance decomposition and connect it to model selection
- [ ] Understand VC dimension as a measure of model capacity
- [ ] Explain the PAC learning framework and sample complexity
- [ ] Visualize why L1 regularization produces sparsity and L2 produces small weights
- [ ] Discuss why overparameterized networks generalize (double descent, implicit regularization)

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, FancyArrowPatch, Ellipse
from matplotlib.colors import LogNorm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(42)

## 1. Gradient Descent Convergence

### Intuitive Explanation

Gradient descent is the workhorse of machine learning optimization. At each step, we move in the
direction of steepest descent:

$$x_{t+1} = x_t - \eta \nabla f(x_t)$$

But how fast does it converge? The answer depends on two things:
1. **The step size** $\eta$ (learning rate) — too large and we overshoot, too small and we crawl
2. **The function's properties** — convex functions converge faster than non-convex ones

**Key convergence rates:**

| Function Type | Rate | What It Means |
|--------------|------|---------------|
| Convex | $O(1/t)$ | Error halves every time you double iterations |
| Strongly convex | $O(e^{-ct})$ | Error decreases exponentially (linear convergence) |
| Non-convex | No guarantee | May get stuck at local minima or saddle points |

The difference is dramatic: for a strongly convex function, 100 iterations might suffice where a merely
convex function needs 10,000.

In [ ]:
def gradient_descent(grad_f, x0, lr, n_steps, f=None):
    """
    Gradient descent with convergence tracking.

    Args:
        grad_f: Function that returns the gradient at x
        x0: Initial point (numpy array)
        lr: Learning rate (step size)
        n_steps: Number of iterations
        f: Optional objective function for tracking loss

    Returns:
        history: dict with 'x' (trajectory), 'f' (function values), 'grad_norm' (gradient norms)
    """
    x = np.array(x0, dtype=float)
    history = {'x': [x.copy()], 'f': [], 'grad_norm': []}

    if f is not None:
        history['f'].append(f(x))

    for t in range(n_steps):
        g = grad_f(x)
        history['grad_norm'].append(np.linalg.norm(g))
        x = x - lr * g
        history['x'].append(x.copy())
        if f is not None:
            history['f'].append(f(x))

    history['x'] = np.array(history['x'])
    return history

# Example: quadratic function f(x) = 0.5 * x^T A x
# Strongly convex when all eigenvalues of A are positive
A_well = np.array([[2.0, 0.0], [0.0, 2.0]])   # condition number = 1
A_ill  = np.array([[10.0, 0.0], [0.0, 1.0]])   # condition number = 10

def make_quadratic(A):
    f = lambda x: 0.5 * x @ A @ x
    grad_f = lambda x: A @ x
    return f, grad_f

x0 = np.array([5.0, 5.0])
print("Well-conditioned (kappa=1): eigenvalues =", np.linalg.eigvalsh(A_well))
print("Ill-conditioned (kappa=10): eigenvalues =", np.linalg.eigvalsh(A_ill))

In [ ]:
# Visualize convergence for different step sizes
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

f_well, grad_well = make_quadratic(A_well)

step_sizes = [0.01, 0.1, 0.45]
titles = ['Too Small (lr=0.01)', 'Just Right (lr=0.1)', 'Too Large (lr=0.45)']
colors = ['blue', 'green', 'red']

for ax, lr, title, color in zip(axes, step_sizes, titles, colors):
    hist = gradient_descent(grad_well, x0, lr, 50, f=f_well)

    # Plot contours
    xx, yy = np.meshgrid(np.linspace(-6, 6, 100), np.linspace(-6, 6, 100))
    zz = np.array([f_well(np.array([xi, yi])) for xi, yi in zip(xx.ravel(), yy.ravel())]).reshape(xx.shape)
    ax.contour(xx, yy, zz, levels=15, alpha=0.5, cmap='coolwarm')

    # Plot trajectory
    traj = hist['x']
    ax.plot(traj[:, 0], traj[:, 1], 'o-', color=color, markersize=3, linewidth=1, alpha=0.7)
    ax.plot(traj[0, 0], traj[0, 1], 'ko', markersize=8, label='Start')
    ax.plot(0, 0, 'r*', markersize=15, label='Optimum')

    ax.set_title(title, fontsize=13)
    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$')
    ax.legend(fontsize=9)
    ax.set_xlim(-6, 6)
    ax.set_ylim(-6, 6)
    ax.set_aspect('equal')

plt.suptitle('Effect of Step Size on Gradient Descent', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Compare convergence rates: convex vs strongly convex
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Strongly convex: f(x) = 0.5 * x^T A x (quadratic)
f_strong, grad_strong = make_quadratic(A_well)
hist_strong = gradient_descent(grad_strong, x0, 0.3, 100, f=f_strong)

# Convex but not strongly convex: f(x) = |x1| + 0.5*x2^2 (approximated with smooth version)
# Use Huber-like: f(x) = sqrt(x1^2 + 0.01) + 0.5*x2^2
def f_convex(x):
    return np.sqrt(x[0]**2 + 0.01) + 0.5 * x[1]**2

def grad_convex(x):
    g1 = x[0] / np.sqrt(x[0]**2 + 0.01)
    g2 = x[1]
    return np.array([g1, g2])

hist_convex = gradient_descent(grad_convex, x0, 0.3, 100, f=f_convex)

# Plot function value convergence
f_star_strong = 0.0
f_star_convex = np.sqrt(0.01)  # minimum value

ax = axes[0]
errors_strong = np.array(hist_strong['f']) - f_star_strong
errors_convex = np.array(hist_convex['f']) - f_star_convex
ax.semilogy(errors_strong + 1e-16, 'b-', linewidth=2, label='Strongly Convex ($O(e^{-ct})$)')
ax.semilogy(errors_convex + 1e-16, 'r-', linewidth=2, label='Convex ($O(1/t)$)')
ax.set_xlabel('Iteration', fontsize=12)
ax.set_ylabel('$f(x_t) - f^*$ (log scale)', fontsize=12)
ax.set_title('Convergence Rate Comparison', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Plot theoretical rates
ax = axes[1]
t = np.arange(1, 101)
ax.semilogy(1.0 / t, 'r--', linewidth=2, label='$O(1/t)$ — Convex')
ax.semilogy(np.exp(-0.1 * t), 'b--', linewidth=2, label='$O(e^{-ct})$ — Strongly Convex')
ax.set_xlabel('Iteration', fontsize=12)
ax.set_ylabel('Error (log scale)', fontsize=12)
ax.set_title('Theoretical Convergence Rates', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("After 100 iterations:")
print(f"  Strongly convex error: {errors_strong[-1]:.2e}")
print(f"  Convex error:          {errors_convex[-1]:.2e}")

### Deep Dive: Choosing the Right Step Size

The optimal step size depends on the function's **Lipschitz constant** $L$ — the maximum curvature of the gradient:

$$\|\nabla f(x) - \nabla f(y)\| \leq L \|x - y\|$$

**The rule:** Set $\eta \leq 1/L$ for guaranteed convergence.

For a quadratic $f(x) = \frac{1}{2}x^T A x$, the Lipschitz constant is $L = \lambda_{\max}(A)$ (the largest eigenvalue).

| Step Size | Behavior | Convergence |
|-----------|----------|-------------|
| $\eta \ll 1/L$ | Very slow, safe | Guaranteed but wastes compute |
| $\eta = 1/L$ | Optimal for GD | Fastest guaranteed convergence |
| $\eta > 2/L$ | Oscillates then diverges | No convergence |

**What this means:** In practice, learning rate is the most important hyperparameter. Too large = training explodes. Too small = training takes forever. Learning rate schedulers (starting large, decreasing over time) try to get the best of both worlds.

---
## 2. Convexity and Guarantees

### Intuitive Explanation

A function is **convex** if every chord lies above the curve — there are no "valleys" to get trapped in. This is the single most important property in optimization because it guarantees that any local minimum is also the *global* minimum.

**Three levels of "niceness":**

1. **Non-convex**: Multiple local minima, saddle points. No guarantees. (Most neural network losses)
2. **Convex**: One global minimum (or a flat region of minima). GD converges at $O(1/t)$.
3. **Strongly convex**: A unique global minimum with curvature bounded away from zero. GD converges exponentially.

**What this means:** Strongly convex is the "easy mode" of optimization. Convex is "medium." Non-convex is "hard mode" — and it's where deep learning lives.

In [ ]:
# Visualize three types of functions
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

x = np.linspace(-3, 3, 300)

# Strongly convex: f(x) = x^2
ax = axes[0]
y = x**2
ax.plot(x, y, 'b-', linewidth=2.5)
ax.fill_between(x, y, alpha=0.1, color='blue')
# Show chord above curve
x1, x2 = -2, 1.5
ax.plot([x1, x2], [x1**2, x2**2], 'r--', linewidth=2, label='Chord (always above)')
ax.plot(0, 0, 'g*', markersize=15, label='Global minimum')
ax.set_title('Strongly Convex: $f(x) = x^2$', fontsize=13)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Convex but not strongly: f(x) = |x|
ax = axes[1]
y = np.abs(x)
ax.plot(x, y, 'b-', linewidth=2.5)
ax.fill_between(x, y, alpha=0.1, color='blue')
x1, x2 = -2, 1.5
ax.plot([x1, x2], [abs(x1), abs(x2)], 'r--', linewidth=2, label='Chord (always above)')
ax.plot(0, 0, 'g*', markersize=15, label='Global minimum')
ax.set_title('Convex: $f(x) = |x|$', fontsize=13)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Non-convex: f(x) = x^4 - 3x^2 + x
ax = axes[2]
y = x**4 - 3*x**2 + 0.5*x
ax.plot(x, y, 'b-', linewidth=2.5)
ax.fill_between(x, y, y.min()-1, alpha=0.1, color='blue')
# Find and mark local minima
from scipy.optimize import minimize_scalar
res1 = minimize_scalar(lambda t: t**4 - 3*t**2 + 0.5*t, bounds=(-2, 0), method='bounded')
res2 = minimize_scalar(lambda t: t**4 - 3*t**2 + 0.5*t, bounds=(0, 2), method='bounded')
ax.plot(res1.x, res1.fun, 'g*', markersize=15, label=f'Global min')
ax.plot(res2.x, res2.fun, 'ro', markersize=10, label=f'Local min')
# Show chord below curve (non-convex!)
x1, x2 = -1.5, 1.5
fx1 = x1**4 - 3*x1**2 + 0.5*x1
fx2 = x2**4 - 3*x2**2 + 0.5*x2
ax.plot([x1, x2], [fx1, fx2], 'r--', linewidth=2, label='Chord (goes below!)')
ax.set_title('Non-convex: $f(x) = x^4 - 3x^2 + 0.5x$', fontsize=13)
ax.set_xlabel('x')
ax.set_ylabel('f(x)')
ax.legend(fontsize=9, loc='upper center')
ax.grid(True, alpha=0.3)

plt.suptitle('Convexity: Why It Matters for Optimization', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

### Deep Dive: Condition Number

The **condition number** $\kappa = \lambda_{\max} / \lambda_{\min}$ measures how "stretched" a function's contours are.

- $\kappa = 1$: Perfect circles — GD goes straight to the minimum
- $\kappa \gg 1$: Elongated ellipses — GD zigzags painfully

**Why it matters in ML:**

| Situation | Effect |
|-----------|--------|
| Poorly scaled features | High condition number, slow training |
| Batch normalization | Reduces effective condition number |
| Adam optimizer | Adapts per-parameter, handles ill-conditioning |
| Feature normalization | Reduces condition number before training |

**Key insight:** The convergence rate of GD on strongly convex functions is $O\left(\left(\frac{\kappa - 1}{\kappa + 1}\right)^t\right)$. When $\kappa = 1$, this is $0^t$ — instant convergence. When $\kappa = 100$, convergence is painfully slow.

In [ ]:
# Visualize effect of condition number on GD paths
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

condition_numbers = [1, 5, 20]
titles = ['$\kappa = 1$ (circular)', '$\kappa = 5$ (moderate)', '$\kappa = 20$ (ill-conditioned)']

for ax, kappa, title in zip(axes, condition_numbers, titles):
    A = np.array([[kappa, 0.0], [0.0, 1.0]])
    f, grad_f = make_quadratic(A)

    # Optimal step size: 2 / (lambda_max + lambda_min)
    lr = 2.0 / (kappa + 1.0)
    hist = gradient_descent(grad_f, np.array([4.0, 4.0]), lr, 50, f=f)

    # Contours
    xx, yy = np.meshgrid(np.linspace(-5, 5, 100), np.linspace(-5, 5, 100))
    zz = np.array([f(np.array([xi, yi])) for xi, yi in zip(xx.ravel(), yy.ravel())]).reshape(xx.shape)
    ax.contour(xx, yy, zz, levels=15, alpha=0.5, cmap='coolwarm')

    # Trajectory
    traj = hist['x']
    ax.plot(traj[:, 0], traj[:, 1], 'b.-', markersize=5, linewidth=1, alpha=0.8)
    ax.plot(traj[0, 0], traj[0, 1], 'ko', markersize=8)
    ax.plot(0, 0, 'r*', markersize=15)

    ax.set_title(title, fontsize=13)
    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$')
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)

plt.suptitle('Condition Number: The Hidden Cost of Ill-Conditioning', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

print("Steps to reach f(x) < 0.01:")
for kappa in condition_numbers:
    A = np.array([[kappa, 0.0], [0.0, 1.0]])
    f, grad_f = make_quadratic(A)
    lr = 2.0 / (kappa + 1.0)
    hist = gradient_descent(grad_f, np.array([4.0, 4.0]), lr, 500, f=f)
    for i, fval in enumerate(hist['f']):
        if fval < 0.01:
            print(f"  kappa={kappa:2d}: {i} steps")
            break
    else:
        print(f"  kappa={kappa:2d}: >500 steps")

---
## 3. Stochastic Gradient Descent

### Intuitive Explanation

In machine learning, the objective is usually a sum over data points:

$$f(w) = \frac{1}{N}\sum_{i=1}^{N} \ell(w; x_i, y_i)$$

**Full gradient descent** computes the gradient using *all* $N$ data points — expensive when $N$ is millions.

**Stochastic gradient descent (SGD)** approximates the gradient using a single random data point (or a small mini-batch). The gradient estimate is noisy but *unbiased*:

$$\mathbb{E}[\nabla \ell(w; x_i, y_i)] = \nabla f(w)$$

**Why stochasticity helps:**
1. **Computational efficiency**: Each step is $O(1)$ instead of $O(N)$
2. **Escaping saddle points**: Noise helps SGD escape flat regions where GD gets stuck
3. **Implicit regularization**: The noise acts as a regularizer, improving generalization

**The tradeoff:** SGD steps are cheaper but noisier. Mini-batches balance this — larger batches reduce variance but cost more compute.

In [ ]:
def sgd(grad_fi, x0, lr, n_steps, n_data, batch_size=1, f=None):
    """
    Stochastic gradient descent with mini-batches.

    Args:
        grad_fi: Function(x, indices) returning gradient estimate from data subset
        x0: Initial point
        lr: Learning rate (can be a function of step t)
        n_steps: Number of iterations
        n_data: Total number of data points
        batch_size: Mini-batch size
        f: Optional full objective function for tracking

    Returns:
        history dict with trajectory and function values
    """
    x = np.array(x0, dtype=float)
    history = {'x': [x.copy()], 'f': []}

    if f is not None:
        history['f'].append(f(x))

    for t in range(n_steps):
        # Sample mini-batch
        indices = np.random.choice(n_data, size=batch_size, replace=False)

        # Learning rate schedule
        current_lr = lr(t) if callable(lr) else lr

        g = grad_fi(x, indices)
        x = x - current_lr * g
        history['x'].append(x.copy())
        if f is not None:
            history['f'].append(f(x))

    history['x'] = np.array(history['x'])
    return history

# Create a synthetic least-squares problem: f(w) = (1/N) sum_i (w^T x_i - y_i)^2
np.random.seed(42)
N = 200
d = 2
X_data = np.random.randn(N, d)
w_true = np.array([3.0, -1.0])
y_data = X_data @ w_true + 0.3 * np.random.randn(N)

def full_loss(w):
    residuals = X_data @ w - y_data
    return np.mean(residuals**2)

def full_grad(w):
    residuals = X_data @ w - y_data
    return 2.0 * X_data.T @ residuals / N

def stochastic_grad(w, indices):
    X_batch = X_data[indices]
    y_batch = y_data[indices]
    residuals = X_batch @ w - y_batch
    return 2.0 * X_batch.T @ residuals / len(indices)

w0 = np.array([0.0, 0.0])
print(f"True weights: {w_true}")
print(f"Initial loss: {full_loss(w0):.4f}")

In [ ]:
# Compare GD vs SGD vs mini-batch SGD
np.random.seed(42)
n_steps = 200

# Full GD
hist_gd = gradient_descent(full_grad, w0, 0.05, n_steps, f=full_loss)

# SGD (batch_size=1)
np.random.seed(42)
hist_sgd1 = sgd(stochastic_grad, w0, 0.01, n_steps, N, batch_size=1, f=full_loss)

# Mini-batch SGD (batch_size=32)
np.random.seed(42)
hist_sgd32 = sgd(stochastic_grad, w0, 0.03, n_steps, N, batch_size=32, f=full_loss)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curves
ax = axes[0]
ax.semilogy(hist_gd['f'], 'b-', linewidth=2, label='Full GD', alpha=0.9)
ax.semilogy(hist_sgd1['f'], 'r-', linewidth=1, label='SGD (batch=1)', alpha=0.6)
ax.semilogy(hist_sgd32['f'], 'g-', linewidth=1.5, label='Mini-batch (batch=32)', alpha=0.7)
ax.set_xlabel('Iteration', fontsize=12)
ax.set_ylabel('Loss (log scale)', fontsize=12)
ax.set_title('Convergence: GD vs SGD', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Trajectories in weight space
ax = axes[1]
ax.plot(hist_gd['x'][:, 0], hist_gd['x'][:, 1], 'b-', linewidth=2, label='Full GD', alpha=0.8)
ax.plot(hist_sgd1['x'][:, 0], hist_sgd1['x'][:, 1], 'r-', linewidth=0.5, label='SGD (batch=1)', alpha=0.4)
ax.plot(hist_sgd32['x'][:, 0], hist_sgd32['x'][:, 1], 'g-', linewidth=1, label='Mini-batch (batch=32)', alpha=0.6)
ax.plot(w_true[0], w_true[1], 'k*', markersize=15, label='True weights', zorder=5)
ax.plot(w0[0], w0[1], 'ko', markersize=8, label='Start')
ax.set_xlabel('$w_1$', fontsize=12)
ax.set_ylabel('$w_2$', fontsize=12)
ax.set_title('Trajectories in Weight Space', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal weights:")
print(f"  Full GD:      [{hist_gd['x'][-1][0]:.4f}, {hist_gd['x'][-1][1]:.4f}]")
print(f"  SGD (B=1):    [{hist_sgd1['x'][-1][0]:.4f}, {hist_sgd1['x'][-1][1]:.4f}]")
print(f"  SGD (B=32):   [{hist_sgd32['x'][-1][0]:.4f}, {hist_sgd32['x'][-1][1]:.4f}]")
print(f"  True:         [{w_true[0]:.4f}, {w_true[1]:.4f}]")

In [ ]:
# Mini-batch size tradeoff
np.random.seed(42)
batch_sizes = [1, 4, 16, 64, N]
n_steps_per = 300

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Adjust learning rates for each batch size
lr_map = {1: 0.005, 4: 0.01, 16: 0.02, 64: 0.04, N: 0.05}

for bs in batch_sizes:
    np.random.seed(42)
    lr = lr_map[bs]
    label = f'B={bs}' if bs < N else f'B={N} (Full GD)'
    if bs == N:
        hist = gradient_descent(full_grad, w0, lr, n_steps_per, f=full_loss)
    else:
        hist = sgd(stochastic_grad, w0, lr, n_steps_per, N, batch_size=bs, f=full_loss)

    # Smooth the SGD curves for visibility
    fvals = np.array(hist['f'])
    if bs < N:
        # Moving average
        window = max(1, 10)
        fvals_smooth = np.convolve(fvals, np.ones(window)/window, mode='valid')
        axes[0].semilogy(fvals_smooth, linewidth=1.5, label=label, alpha=0.8)
    else:
        axes[0].semilogy(fvals, linewidth=2.5, label=label, alpha=0.9)

axes[0].set_xlabel('Iteration', fontsize=12)
axes[0].set_ylabel('Loss (log scale, smoothed)', fontsize=12)
axes[0].set_title('Convergence vs Batch Size', fontsize=13)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Variance of gradient estimates
ax = axes[1]
batch_range = [1, 2, 4, 8, 16, 32, 64, 128]
grad_variances = []
w_test = np.array([1.0, 1.0])

for bs in batch_range:
    grads = []
    for _ in range(200):
        idx = np.random.choice(N, size=bs, replace=False)
        g = stochastic_grad(w_test, idx)
        grads.append(g)
    grads = np.array(grads)
    grad_variances.append(np.mean(np.var(grads, axis=0)))

ax.loglog(batch_range, grad_variances, 'bo-', linewidth=2, markersize=8)
ax.loglog(batch_range, grad_variances[0] / np.array(batch_range), 'r--', linewidth=1.5, label='$O(1/B)$ scaling')
ax.set_xlabel('Batch Size', fontsize=12)
ax.set_ylabel('Gradient Variance', fontsize=12)
ax.set_title('Gradient Variance vs Batch Size', fontsize=13)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key insight: Gradient variance scales as O(1/B)")
print("Doubling batch size halves variance but doubles compute per step")

### Deep Dive: Why SGD Works So Well in Practice

SGD's noise is not just a computational shortcut — it's a *feature*:

1. **Escaping sharp minima**: SGD's noise bounces it out of sharp, narrow minima and into flatter ones. Flat minima tend to generalize better (the loss doesn't change much if weights shift slightly).

2. **Implicit regularization**: SGD with a finite learning rate implicitly prefers solutions with smaller norms. This acts like free regularization.

3. **Exploration**: Early in training, large noise helps explore the loss landscape. Late in training, we want less noise — hence learning rate decay.

| Technique | Purpose | Effect |
|-----------|---------|--------|
| Learning rate warmup | Avoid early instability | Start small, increase to target LR |
| Learning rate decay | Converge precisely | Reduce noise late in training |
| Momentum | Smooth out noise | Average gradients over time |
| Adam | Adapt per-parameter | Handle different scales automatically |

**Common misconception:** "SGD is just a noisy version of GD." *Reality:* SGD finds qualitatively different (often better) solutions than GD because the noise structure matters.

---
## 4. The Bias-Variance Tradeoff

### Intuitive Explanation

When we train a model, we want it to generalize — to perform well on *new* data, not just the training data. The **bias-variance decomposition** tells us exactly what can go wrong:

$$\text{Expected Error} = \text{Bias}^2 + \text{Variance} + \text{Irreducible Noise}$$

#### Breaking down the formula:

| Component | Meaning | Caused By |
|-----------|---------|-----------|
| **Bias**$^2$ | How far off the average prediction is from truth | Model too simple (underfitting) |
| **Variance** | How much predictions change across different training sets | Model too complex (overfitting) |
| **Irreducible noise** | Inherent randomness in the data | Nothing — this is the floor |

**What this means:** You can't minimize both bias and variance simultaneously. Simple models have high bias but low variance. Complex models have low bias but high variance. The sweet spot is in the middle.

In [ ]:
# Demonstrate bias-variance tradeoff with polynomial regression
np.random.seed(42)

# True function
def true_function(x):
    return np.sin(2 * x) + 0.5 * np.cos(3 * x)

# Generate multiple training sets and fit models of different complexity
n_train = 25
n_datasets = 100
noise_std = 0.4
x_test = np.linspace(0, 2*np.pi, 200)
y_true = true_function(x_test)

degrees = [1, 3, 5, 9, 15]
predictions = {d: [] for d in degrees}

for _ in range(n_datasets):
    x_train = np.random.uniform(0, 2*np.pi, n_train)
    y_train = true_function(x_train) + noise_std * np.random.randn(n_train)

    for d in degrees:
        poly = PolynomialFeatures(d)
        X_train_poly = poly.fit_transform(x_train.reshape(-1, 1))
        X_test_poly = poly.transform(x_test.reshape(-1, 1))

        model = LinearRegression()
        model.fit(X_train_poly, y_train)
        y_pred = model.predict(X_test_poly)
        predictions[d].append(y_pred)

# Compute bias^2 and variance
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for idx, d in enumerate(degrees[:5]):
    row, col = divmod(idx, 3)
    ax = axes[row][col]

    preds = np.array(predictions[d])
    mean_pred = preds.mean(axis=0)

    # Plot individual predictions (sample of 20)
    for i in range(min(20, n_datasets)):
        ax.plot(x_test, preds[i], 'b-', alpha=0.1, linewidth=0.5)

    ax.plot(x_test, y_true, 'r-', linewidth=2.5, label='True function')
    ax.plot(x_test, mean_pred, 'g--', linewidth=2, label='Mean prediction')

    bias_sq = np.mean((mean_pred - y_true)**2)
    variance = np.mean(np.var(preds, axis=0))

    ax.set_title(f'Degree {d}: Bias²={bias_sq:.3f}, Var={variance:.3f}', fontsize=11)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.legend(fontsize=8, loc='upper right')
    ax.set_ylim(-3, 3)
    ax.grid(True, alpha=0.3)

# Use last subplot for the decomposition summary
ax = axes[1][2]
biases_sq = []
variances = []
for d in degrees:
    preds = np.array(predictions[d])
    mean_pred = preds.mean(axis=0)
    biases_sq.append(np.mean((mean_pred - y_true)**2))
    variances.append(np.mean(np.var(preds, axis=0)))

total = np.array(biases_sq) + np.array(variances) + noise_std**2
ax.plot(degrees, biases_sq, 'bo-', linewidth=2, label='Bias²', markersize=8)
ax.plot(degrees, variances, 'rs-', linewidth=2, label='Variance', markersize=8)
ax.plot(degrees, total, 'g^-', linewidth=2, label='Total Error', markersize=8)
ax.axhline(y=noise_std**2, color='gray', linestyle=':', linewidth=1.5, label=f'Noise floor ({noise_std**2:.2f})')
ax.set_xlabel('Polynomial Degree', fontsize=12)
ax.set_ylabel('Error', fontsize=12)
ax.set_title('Bias-Variance Decomposition', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.suptitle('Bias-Variance Tradeoff: Model Complexity vs Generalization', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# The classic U-shaped test error curve
np.random.seed(42)
n_train = 30
n_test = 200

x_train = np.random.uniform(0, 2*np.pi, n_train)
y_train = true_function(x_train) + noise_std * np.random.randn(n_train)
x_test_pts = np.random.uniform(0, 2*np.pi, n_test)
y_test = true_function(x_test_pts) + noise_std * np.random.randn(n_test)

degrees_range = range(1, 20)
train_errors = []
test_errors = []

for d in degrees_range:
    poly = PolynomialFeatures(d)
    X_tr = poly.fit_transform(x_train.reshape(-1, 1))
    X_te = poly.transform(x_test_pts.reshape(-1, 1))

    model = LinearRegression()
    model.fit(X_tr, y_train)

    train_errors.append(np.mean((model.predict(X_tr) - y_train)**2))
    test_errors.append(np.mean((model.predict(X_te) - y_test)**2))

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(list(degrees_range), train_errors, 'b-o', linewidth=2, markersize=6, label='Training Error')
ax.plot(list(degrees_range), test_errors, 'r-s', linewidth=2, markersize=6, label='Test Error')
ax.axhline(y=noise_std**2, color='gray', linestyle=':', linewidth=1.5, label=f'Irreducible noise ({noise_std**2:.2f})')

# Mark the sweet spot
best_deg = list(degrees_range)[np.argmin(test_errors)]
ax.axvline(x=best_deg, color='green', linestyle='--', linewidth=1.5, alpha=0.7, label=f'Best complexity (degree={best_deg})')

# Annotations
ax.annotate('Underfitting\n(high bias)', xy=(2, test_errors[1]), fontsize=11, color='orange',
            xytext=(3.5, test_errors[1]+0.3), arrowprops=dict(arrowstyle='->', color='orange'))
ax.annotate('Overfitting\n(high variance)', xy=(15, min(test_errors[-5:])+0.5), fontsize=11, color='orange',
            xytext=(13, min(test_errors[-5:])+1.0), arrowprops=dict(arrowstyle='->', color='orange'))

ax.set_xlabel('Model Complexity (Polynomial Degree)', fontsize=12)
ax.set_ylabel('Mean Squared Error', fontsize=12)
ax.set_title('The U-Shaped Test Error Curve', fontsize=14)
ax.legend(fontsize=11)
ax.set_ylim(0, max(test_errors[0], 3))
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Best polynomial degree: {best_deg}")
print(f"Train error at best: {train_errors[best_deg-1]:.4f}")
print(f"Test error at best:  {test_errors[best_deg-1]:.4f}")

---
## 5. VC Dimension and Generalization

### Intuitive Explanation

How do we measure a model's **capacity** — its ability to fit arbitrary patterns? The **Vapnik-Chervonenkis (VC) dimension** provides a formal answer.

**Definition:** The VC dimension of a model class is the largest number of points it can **shatter** — i.e., classify correctly for *every possible* labeling of those points.

**Examples:**

| Model | VC Dimension | Intuition |
|-------|-------------|-----------|
| Linear classifier in $\mathbb{R}^d$ | $d + 1$ | Can shatter $d+1$ points in general position |
| Linear classifier in $\mathbb{R}^2$ | 3 | Can shatter any 3 non-collinear points |
| Polynomial of degree $k$ | $k + 1$ | Can memorize $k+1$ points exactly |
| Neural net with $W$ weights | $O(W \log W)$ | Roughly proportional to parameter count |
| $k$-nearest neighbors | $\infty$ | Can memorize any dataset (but doesn't generalize!) |

**What this means:** Higher VC dimension = more expressive model = needs more data to generalize reliably.

In [ ]:
# Visualize shattering: a linear classifier in R^2 can shatter 3 points but not 4

fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# 3 points: show all 8 labelings can be separated
points_3 = np.array([[0, 0], [2, 0], [1, 2]])
labelings_3 = [
    [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1],
    [1, 1, 0], [1, 0, 1], [0, 1, 1], [1, 1, 1]
]

for i, (ax, labels) in enumerate(zip(axes[0], labelings_3[:4])):
    for j, (pt, lab) in enumerate(zip(points_3, labels)):
        color = 'blue' if lab == 1 else 'red'
        marker = 'o' if lab == 1 else 'x'
        ax.scatter(pt[0], pt[1], c=color, marker=marker, s=150, zorder=5, edgecolors='black', linewidths=1)

    ax.set_xlim(-0.5, 3)
    ax.set_ylim(-0.5, 3)
    ax.set_title(f'Labeling {labels}', fontsize=10)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    if i == 0:
        ax.set_ylabel('VC dim = 3\n(3 points shattered)', fontsize=11)

for i, (ax, labels) in enumerate(zip(axes[1][:4], labelings_3[4:])):
    for j, (pt, lab) in enumerate(zip(points_3, labels)):
        color = 'blue' if lab == 1 else 'red'
        marker = 'o' if lab == 1 else 'x'
        ax.scatter(pt[0], pt[1], c=color, marker=marker, s=150, zorder=5, edgecolors='black', linewidths=1)

    ax.set_xlim(-0.5, 3)
    ax.set_ylim(-0.5, 3)
    ax.set_title(f'Labeling {labels}', fontsize=10)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)

axes[0][0].annotate('All 8 labelings\nseparable by a line!',
                      xy=(1.5, 2.5), fontsize=10, color='green',
                      ha='center', fontweight='bold')

plt.suptitle('VC Dimension: Shattering 3 Points with a Linear Classifier', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print("A linear classifier in R^2 has VC dimension = 3")
print("It can shatter (perfectly classify) any 3 points in general position")
print("But there exist 4 points (e.g., XOR pattern) that NO line can shatter")

### Deep Dive: The VC Generalization Bound

The VC dimension connects model complexity to generalization through a fundamental bound:

$$\text{Test Error} \leq \text{Training Error} + O\left(\sqrt{\frac{d_{VC} \log(N/d_{VC})}{N}}\right)$$

where $d_{VC}$ is the VC dimension and $N$ is the number of training samples.

**What this tells us:**
- The gap between training and test error grows with $d_{VC}$ (model complexity)
- The gap shrinks with $N$ (more data helps)
- To keep the gap small: need $N \gg d_{VC}$

**Key insight:** This is the theoretical justification for the rule of thumb "you need at least 10x more data points than parameters."

#### Common Misconceptions

| Misconception | Reality |
|---------------|---------|
| "Low VC dim = good model" | Low VC dim means simple model — might underfit |
| "VC bounds are tight" | VC bounds are usually very loose; they give qualitative, not quantitative guidance |
| "More parameters = higher VC dim always" | Regularization can effectively reduce VC dimension |

---
## 6. PAC Learning

### Intuitive Explanation

**Probably Approximately Correct (PAC)** learning asks: "How much data do we need to *probably* find a model that is *approximately* correct?"

More precisely, for given:
- $\epsilon$ = accuracy tolerance ("approximately correct" — error < $\epsilon$)
- $\delta$ = confidence tolerance ("probably" — succeed with probability $\geq 1 - \delta$)

PAC learning tells us the **sample complexity**: the minimum number of training examples $N$ needed.

**For a finite hypothesis class** $\mathcal{H}$ with $|\mathcal{H}|$ hypotheses:

$$N \geq \frac{1}{\epsilon}\left(\ln |\mathcal{H}| + \ln \frac{1}{\delta}\right)$$

**What this means in plain English:**
- Want more accuracy (smaller $\epsilon$)? Need more data.
- Want more confidence (smaller $\delta$)? Need more data (but only logarithmically).
- More complex model (larger $|\mathcal{H}|$)? Need more data (also logarithmically).

In [ ]:
# Visualize PAC sample complexity
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Sample complexity vs epsilon
ax = axes[0]
epsilon = np.linspace(0.01, 0.5, 100)
for H_size, color, label in [(10, 'blue', '|H|=10'), (100, 'red', '|H|=100'), (1000, 'green', '|H|=1000')]:
    delta = 0.05
    N_pac = (1/epsilon) * (np.log(H_size) + np.log(1/delta))
    ax.plot(epsilon, N_pac, color=color, linewidth=2, label=label)
ax.set_xlabel('$\epsilon$ (error tolerance)', fontsize=12)
ax.set_ylabel('Sample complexity $N$', fontsize=12)
ax.set_title('More accuracy → more data', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Plot 2: Sample complexity vs delta
ax = axes[1]
delta = np.linspace(0.001, 0.5, 100)
for eps, color, label in [(0.1, 'blue', 'ε=0.1'), (0.05, 'red', 'ε=0.05'), (0.01, 'green', 'ε=0.01')]:
    H_size = 100
    N_pac = (1/eps) * (np.log(H_size) + np.log(1/delta))
    ax.plot(delta, N_pac, color=color, linewidth=2, label=label)
ax.set_xlabel('$\delta$ (failure probability)', fontsize=12)
ax.set_ylabel('Sample complexity $N$', fontsize=12)
ax.set_title('More confidence → more data (log)', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Plot 3: Sample complexity vs hypothesis class size
ax = axes[2]
H_sizes = np.logspace(1, 6, 100)
for eps, color, label in [(0.1, 'blue', 'ε=0.1'), (0.05, 'red', 'ε=0.05'), (0.01, 'green', 'ε=0.01')]:
    delta = 0.05
    N_pac = (1/eps) * (np.log(H_sizes) + np.log(1/delta))
    ax.semilogx(H_sizes, N_pac, color=color, linewidth=2, label=label)
ax.set_xlabel('$|\mathcal{H}|$ (hypothesis class size)', fontsize=12)
ax.set_ylabel('Sample complexity $N$', fontsize=12)
ax.set_title('More complex model → more data (log)', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.suptitle('PAC Learning: How Much Data Do You Need?', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

# Concrete example
epsilon = 0.05
delta = 0.05
H_size = 1000
N_required = int(np.ceil((1/epsilon) * (np.log(H_size) + np.log(1/delta))))
print(f"Example: To achieve {epsilon:.0%} error with {1-delta:.0%} confidence")
print(f"  with |H| = {H_size} hypotheses, you need N >= {N_required} samples")

### Deep Dive: What PAC Learning Tells Us About ML

PAC learning provides the theoretical foundation for understanding *learnability*:

**A concept class is PAC-learnable** if there exists an algorithm that, for any $\epsilon > 0$ and $\delta > 0$, can learn a hypothesis with error $\leq \epsilon$ with probability $\geq 1 - \delta$, using a number of samples polynomial in $1/\epsilon$, $1/\delta$, and the model size.

**Key takeaways for practitioners:**

1. **Logarithmic dependence on $|\mathcal{H}|$ and $1/\delta$** — making your model 10x more complex only increases data needs by $\ln(10) \approx 2.3$. That's surprisingly cheap!

2. **Linear dependence on $1/\epsilon$** — going from 90% to 99% accuracy is much harder than 50% to 90%.

3. **Connection to VC dimension**: For infinite hypothesis classes (like neural networks), replace $\ln|\mathcal{H}|$ with $d_{VC}$ to get similar bounds.

**Why this matters:** PAC theory tells us that learning *is* possible with finite data — not something obvious from first principles. It also tells us the fundamental resource tradeoffs.

---
## 7. Regularization Theory

### Intuitive Explanation

Regularization adds a penalty to the loss function to prevent overfitting:

$$\text{Total Loss} = \text{Data Loss} + \lambda \cdot \text{Penalty}(w)$$

The two most common penalties:
- **L1 (Lasso):** $\|w\|_1 = \sum |w_i|$ — produces **sparse** solutions (many weights exactly zero)
- **L2 (Ridge):** $\|w\|_2^2 = \sum w_i^2$ — produces **small** weights (but rarely exactly zero)

**Why does L1 give sparsity?** It's all about geometry. The L1 constraint region is a *diamond*, and the loss contours typically hit the diamond at a corner — where some coordinates are exactly zero. The L2 constraint region is a *sphere*, which has no corners, so the intersection point rarely has exact zeros.

In [ ]:
# The geometry of L1 vs L2 regularization
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Create elliptical contours (represent loss function)
theta_range = np.linspace(0, 2*np.pi, 200)

# Shifted center for the loss function
center = np.array([2.0, 1.5])

# L1 constraint: |w1| + |w2| <= t
def l1_boundary(t):
    pts = []
    for th in np.linspace(0, 2*np.pi, 200):
        x = t * np.cos(th)
        y = t * np.sin(th)
        # Project onto L1 ball
        scale = (np.abs(x) + np.abs(y))
        if scale > 0:
            x, y = x * t / scale, y * t / scale
        pts.append([x, y])
    return np.array(pts)

# L2 constraint: w1^2 + w2^2 <= t^2
def l2_boundary(t):
    return np.column_stack([t * np.cos(theta_range), t * np.sin(theta_range)])

# Plot L1
ax = axes[0]
A_loss = np.array([[1.0, 0.3], [0.3, 2.0]])
xx, yy = np.meshgrid(np.linspace(-3, 4, 200), np.linspace(-3, 4, 200))
zz = np.zeros_like(xx)
for i in range(xx.shape[0]):
    for j in range(xx.shape[1]):
        d = np.array([xx[i,j], yy[i,j]]) - center
        zz[i,j] = 0.5 * d @ A_loss @ d

ax.contour(xx, yy, zz, levels=10, cmap='RdYlBu_r', alpha=0.6)

# Diamond (L1 ball)
t = 1.5
diamond = np.array([[t, 0], [0, t], [-t, 0], [0, -t], [t, 0]])
ax.fill(diamond[:, 0], diamond[:, 1], alpha=0.2, color='blue')
ax.plot(diamond[:, 0], diamond[:, 1], 'b-', linewidth=2)

# Mark the solution (corner of diamond)
ax.plot(0, t, 'g*', markersize=20, zorder=5, label='L1 solution (sparse!)')
ax.plot(center[0], center[1], 'r*', markersize=15, label='Unconstrained optimum')

ax.set_title('L1 (Lasso): Diamond Constraint', fontsize=13)
ax.set_xlabel('$w_1$', fontsize=12)
ax.set_ylabel('$w_2$', fontsize=12)
ax.set_xlim(-3, 4)
ax.set_ylim(-3, 4)
ax.set_aspect('equal')
ax.legend(fontsize=9, loc='lower right')
ax.grid(True, alpha=0.3)

# Plot L2
ax = axes[1]
ax.contour(xx, yy, zz, levels=10, cmap='RdYlBu_r', alpha=0.6)

# Circle (L2 ball)
circle_pts = l2_boundary(t)
ax.fill(circle_pts[:, 0], circle_pts[:, 1], alpha=0.2, color='red')
ax.plot(circle_pts[:, 0], circle_pts[:, 1], 'r-', linewidth=2)

# Mark the solution (on circle, not at corner)
# Approximate L2-constrained solution
w_l2 = center / np.linalg.norm(center) * t
ax.plot(w_l2[0], w_l2[1], 'g*', markersize=20, zorder=5, label='L2 solution (small)')
ax.plot(center[0], center[1], 'r*', markersize=15, label='Unconstrained optimum')

ax.set_title('L2 (Ridge): Circle Constraint', fontsize=13)
ax.set_xlabel('$w_1$', fontsize=12)
ax.set_ylabel('$w_2$', fontsize=12)
ax.set_xlim(-3, 4)
ax.set_ylim(-3, 4)
ax.set_aspect('equal')
ax.legend(fontsize=9, loc='lower right')
ax.grid(True, alpha=0.3)

# Plot comparison of weight distributions
ax = axes[2]
np.random.seed(42)
n_features = 20
X_reg = np.random.randn(100, n_features)
w_true_reg = np.zeros(n_features)
w_true_reg[:5] = [3, -2, 1.5, -1, 0.5]  # Only 5 features matter
y_reg = X_reg @ w_true_reg + 0.3 * np.random.randn(100)

# Fit models
from sklearn.linear_model import Lasso, Ridge
lasso = Lasso(alpha=0.1).fit(X_reg, y_reg)
ridge = Ridge(alpha=1.0).fit(X_reg, y_reg)

x_pos = np.arange(n_features)
width = 0.35
ax.bar(x_pos - width/2, np.abs(lasso.coef_), width, color='blue', alpha=0.7, label='L1 (Lasso)')
ax.bar(x_pos + width/2, np.abs(ridge.coef_), width, color='red', alpha=0.7, label='L2 (Ridge)')
ax.axvline(x=4.5, color='green', linestyle='--', linewidth=1.5, alpha=0.5, label='True sparse boundary')
ax.set_xlabel('Feature Index', fontsize=12)
ax.set_ylabel('|Weight|', fontsize=12)
ax.set_title('Weight Magnitudes: L1 vs L2', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('Why L1 Gives Sparsity and L2 Gives Small Weights', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

print(f"L1 (Lasso) non-zero weights: {np.sum(np.abs(lasso.coef_) > 0.01)}/{n_features}")
print(f"L2 (Ridge) non-zero weights: {np.sum(np.abs(ridge.coef_) > 0.01)}/{n_features}")

In [ ]:
# Regularization paths: how weights change with lambda
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

alphas = np.logspace(-3, 2, 100)

# Lasso path
lasso_coefs = []
for alpha in alphas:
    model = Lasso(alpha=alpha, max_iter=10000)
    model.fit(X_reg, y_reg)
    lasso_coefs.append(model.coef_.copy())
lasso_coefs = np.array(lasso_coefs)

ax = axes[0]
for j in range(n_features):
    color = 'blue' if j < 5 else 'gray'
    alpha_val = 0.9 if j < 5 else 0.3
    linewidth = 2 if j < 5 else 0.5
    label = f'Feature {j}' if j < 5 else (None if j > 5 else 'Noise features')
    ax.semilogx(alphas, lasso_coefs[:, j], color=color, alpha=alpha_val, linewidth=linewidth, label=label)
ax.set_xlabel('$\lambda$ (regularization strength)', fontsize=12)
ax.set_ylabel('Weight value', fontsize=12)
ax.set_title('L1 Regularization Path (Lasso)', fontsize=13)
ax.legend(fontsize=8, loc='upper right')
ax.grid(True, alpha=0.3)

# Ridge path
ridge_coefs = []
for alpha in alphas:
    model = Ridge(alpha=alpha)
    model.fit(X_reg, y_reg)
    ridge_coefs.append(model.coef_.copy())
ridge_coefs = np.array(ridge_coefs)

ax = axes[1]
for j in range(n_features):
    color = 'red' if j < 5 else 'gray'
    alpha_val = 0.9 if j < 5 else 0.3
    linewidth = 2 if j < 5 else 0.5
    label = f'Feature {j}' if j < 5 else (None if j > 5 else 'Noise features')
    ax.semilogx(alphas, ridge_coefs[:, j], color=color, alpha=alpha_val, linewidth=linewidth, label=label)
ax.set_xlabel('$\lambda$ (regularization strength)', fontsize=12)
ax.set_ylabel('Weight value', fontsize=12)
ax.set_title('L2 Regularization Path (Ridge)', fontsize=13)
ax.legend(fontsize=8, loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key observation:")
print("  L1: Weights drop to EXACTLY zero as lambda increases (feature selection!)")
print("  L2: Weights shrink smoothly toward zero but never reach it")

### Deep Dive: Regularization as Bayesian Priors

There's a beautiful connection between regularization and Bayesian statistics:

| Regularizer | Equivalent Prior | Distribution |
|------------|-----------------|--------------|
| L2 (Ridge) | Gaussian prior: $w_i \sim \mathcal{N}(0, 1/\lambda)$ | Weights cluster near zero |
| L1 (Lasso) | Laplace prior: $w_i \sim \text{Laplace}(0, 1/\lambda)$ | Weights sparse (peak at zero) |
| Elastic Net | Mix of Gaussian + Laplace | Best of both worlds |

**What this means:** When you add L2 regularization, you're saying "I believe the weights are probably small." When you add L1, you're saying "I believe most weights are probably zero." The strength $\lambda$ controls how strongly you hold this belief.

**Elastic Net** combines both:
$$\text{Penalty} = \alpha \|w\|_1 + (1-\alpha) \|w\|_2^2$$

This gives sparsity (from L1) while handling correlated features better (from L2).

#### Key Insight

Regularization is not a "trick" — it's a principled way of encoding prior knowledge about what good solutions look like. Every regularizer implicitly answers the question: "What kind of models do I expect to work well?" 

---
## 8. Why Overparameterized Networks Generalize

### Intuitive Explanation

Classical learning theory says: more parameters → more overfitting. But modern neural networks have *millions* of parameters (far more than training examples) and still generalize beautifully. This contradicts classical theory and is one of the deepest open questions in ML.

**The double descent phenomenon:** As model complexity increases, test error follows a U-shape (classical regime), but then *decreases again* after the interpolation threshold (where the model can perfectly fit the training data).

Three key ideas explain why overparameterization works:

1. **Double descent**: The classical bias-variance tradeoff is incomplete — it misses the "modern" regime
2. **Implicit regularization**: SGD, by its nature, finds "simple" solutions among the many that fit the data
3. **Lottery ticket hypothesis**: Large networks contain small subnetworks that do most of the work

In [ ]:
# Simulate the double descent curve
np.random.seed(42)

# Generate data from a moderately complex function
n_total = 40
x_all = np.linspace(0, 1, n_total)
y_true_fn = lambda x: np.sin(4 * np.pi * x) + 0.5 * np.cos(6 * np.pi * x)
noise_level = 0.5

n_train = 25
indices = np.random.choice(n_total, n_train, replace=False)
x_train_dd = x_all[indices]
y_train_dd = y_true_fn(x_train_dd) + noise_level * np.random.randn(n_train)
mask = np.ones(n_total, dtype=bool)
mask[indices] = False
x_test_dd = x_all[mask]
y_test_dd = y_true_fn(x_test_dd) + noise_level * np.random.randn(n_total - n_train)

# Fit polynomials of increasing degree, including overparameterized
degrees_dd = list(range(1, 35))
train_errors_dd = []
test_errors_dd = []

for d in degrees_dd:
    poly = PolynomialFeatures(d)
    X_tr = poly.fit_transform(x_train_dd.reshape(-1, 1))
    X_te = poly.transform(x_test_dd.reshape(-1, 1))

    # Use Ridge with tiny regularization for numerical stability past interpolation
    from sklearn.linear_model import Ridge
    model = Ridge(alpha=1e-10)
    model.fit(X_tr, y_train_dd)

    y_pred_tr = model.predict(X_tr)
    y_pred_te = model.predict(X_te)

    train_errors_dd.append(np.mean((y_pred_tr - y_train_dd)**2))
    test_errors_dd.append(np.clip(np.mean((y_pred_te - y_test_dd)**2), 0, 15))

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(degrees_dd, train_errors_dd, 'b-o', linewidth=2, markersize=4, label='Training Error', alpha=0.8)
ax.plot(degrees_dd, test_errors_dd, 'r-s', linewidth=2, markersize=4, label='Test Error', alpha=0.8)
ax.axhline(y=noise_level**2, color='gray', linestyle=':', linewidth=1.5, label=f'Noise floor ({noise_level**2:.2f})')
ax.axvline(x=n_train, color='green', linestyle='--', linewidth=2, alpha=0.6,
           label=f'Interpolation threshold (d={n_train})')

# Annotate regions
ax.annotate('Classical\nregime', xy=(8, 1), fontsize=12, color='blue',
            ha='center', fontweight='bold')
ax.annotate('Modern\n(overparameterized)\nregime', xy=(30, 1), fontsize=12, color='red',
            ha='center', fontweight='bold')
ax.annotate('Interpolation\npeak', xy=(n_train, max(test_errors_dd)), fontsize=10, color='green',
            ha='center', va='bottom')

ax.set_xlabel('Model Complexity (Polynomial Degree)', fontsize=13)
ax.set_ylabel('Mean Squared Error', fontsize=13)
ax.set_title('Double Descent: Beyond the Classical Bias-Variance Tradeoff', fontsize=14)
ax.legend(fontsize=11, loc='upper right')
ax.set_ylim(0, min(15, max(test_errors_dd) * 1.1))
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Double descent: test error peaks at the interpolation threshold,")
print("then DECREASES as model becomes even more overparameterized!")

In [ ]:
# Demonstrate implicit regularization: SGD finds minimum-norm solutions
np.random.seed(42)

# Underdetermined system: more parameters than data points
n_data_imp = 20
n_params = 100  # Way more parameters than data

X_imp = np.random.randn(n_data_imp, n_params)
w_target = np.zeros(n_params)
w_target[:5] = np.array([2, -1, 0.5, 1.5, -0.8])  # Sparse true weights
y_imp = X_imp @ w_target + 0.1 * np.random.randn(n_data_imp)

# Many solutions exist! Let's see what different methods find.

# Method 1: Minimum L2-norm solution (pseudoinverse)
w_minnorm = X_imp.T @ np.linalg.solve(X_imp @ X_imp.T, y_imp)

# Method 2: SGD from zero initialization (implicit regularization)
w_sgd = np.zeros(n_params)
lr_imp = 0.001
for epoch in range(1000):
    for i in np.random.permutation(n_data_imp):
        xi = X_imp[i:i+1]
        yi = y_imp[i:i+1]
        grad = 2 * xi.T @ (xi @ w_sgd - yi)
        w_sgd = w_sgd - lr_imp * grad.flatten()

# Method 3: Random solution (also fits data but has large norm)
# Find a particular solution + null space component
w_particular = w_minnorm
null_component = np.random.randn(n_params)
# Project onto null space of X
proj = X_imp.T @ np.linalg.solve(X_imp @ X_imp.T, X_imp)
null_component = null_component - proj @ null_component
w_random = w_particular + 3 * null_component  # Add large null space component

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

methods = [
    ('Min-norm (Pseudoinverse)', w_minnorm, 'blue'),
    ('SGD from Zero Init', w_sgd, 'green'),
    ('Random Solution', w_random, 'red')
]

for ax, (name, w, color) in zip(axes, methods):
    ax.bar(range(n_params), w, color=color, alpha=0.7, width=1.0)
    ax.set_xlabel('Parameter Index', fontsize=11)
    ax.set_ylabel('Weight Value', fontsize=11)
    train_err = np.mean((X_imp @ w - y_imp)**2)
    w_norm = np.linalg.norm(w)
    ax.set_title(f'{name}\nTrain MSE={train_err:.4f}, ||w||={w_norm:.1f}', fontsize=11)
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim(-4, 4)

plt.suptitle('Implicit Regularization: All Solutions Fit Data, But SGD Finds the Simple One',
             fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print(f"Solution norms:")
print(f"  Min-norm:  ||w|| = {np.linalg.norm(w_minnorm):.4f}")
print(f"  SGD:       ||w|| = {np.linalg.norm(w_sgd):.4f}")
print(f"  Random:    ||w|| = {np.linalg.norm(w_random):.4f}")
print(f"\nAll have near-zero training error — but the norms are very different!")
print(f"SGD implicitly finds a solution close to the minimum-norm solution.")

### Deep Dive: The Lottery Ticket Hypothesis

The **Lottery Ticket Hypothesis** (Frankle & Carlin, 2019) proposes:

> A randomly initialized, dense neural network contains a subnetwork (a "winning ticket") that — when trained in isolation — reaches comparable accuracy to the full network in a similar number of training steps.

**What this means:**
- Large networks work well not because all parameters are needed, but because having many parameters increases the chance of containing a good subnetwork
- You can prune 90%+ of weights after training with minimal accuracy loss
- The initial random values of the winning ticket matter — it's the specific initialization that makes it trainable

**Why this is profound:**
1. It suggests most parameters in large networks are "wasted" at inference time
2. It explains why overparameterization helps *training* even if not needed for *representation*
3. It motivates network pruning and efficient inference

#### Open Questions (Honest Assessment)

| Question | Current Understanding |
|----------|---------------------|
| Why does SGD find flat minima? | Partially understood — noise scale matters |
| Is double descent universal? | Observed broadly, but theory is incomplete |
| Can we find winning tickets cheaply? | Active research; training is still needed to identify them |
| Why do large models generalize at all? | Multiple competing theories; no consensus yet |

**The honest truth:** We don't fully understand why deep learning works as well as it does. The theory is catching up to the practice, and that's what makes this field exciting.

---
## Exercises

### Exercise 1: Implement Gradient Descent with Momentum

Momentum accelerates GD by accumulating a velocity vector:

$$v_{t+1} = \beta v_t + \nabla f(x_t)$$
$$x_{t+1} = x_t - \eta v_{t+1}$$

Implement this and compare convergence with vanilla GD on an ill-conditioned quadratic.

In [ ]:
# EXERCISE 1: Implement gradient descent with momentum
def gradient_descent_momentum(grad_f, x0, lr, n_steps, beta=0.9, f=None):
    """
    Gradient descent with momentum.

    Args:
        grad_f: Gradient function
        x0: Initial point
        lr: Learning rate
        n_steps: Number of iterations
        beta: Momentum coefficient (default 0.9)
        f: Optional objective function for tracking

    Returns:
        history dict with trajectory and function values
    """
    x = np.array(x0, dtype=float)
    v = np.zeros_like(x)  # Initialize velocity
    history = {'x': [x.copy()], 'f': []}

    if f is not None:
        history['f'].append(f(x))

    for t in range(n_steps):
        # TODO: Implement momentum update
        # Hint: First update velocity, then update position
        # v = beta * v + grad_f(x)
        # x = x - lr * v

        pass  # Replace with your implementation

        history['x'].append(x.copy())
        if f is not None:
            history['f'].append(f(x))

    history['x'] = np.array(history['x'])
    return history

# Test on ill-conditioned quadratic
A_test = np.array([[20.0, 0.0], [0.0, 1.0]])  # condition number = 20
f_test, grad_test = make_quadratic(A_test)
x0_test = np.array([5.0, 5.0])

# Test your implementation
hist_mom = gradient_descent_momentum(grad_test, x0_test, lr=0.05, n_steps=100, beta=0.9, f=f_test)
hist_vanilla = gradient_descent(grad_test, x0_test, lr=0.05, n_steps=100, f=f_test)

# Verify
if len(hist_mom['f']) > 1 and hist_mom['f'][-1] != hist_mom['f'][0]:
    print(f"Momentum final loss:  {hist_mom['f'][-1]:.6f}")
    print(f"Vanilla final loss:   {hist_vanilla['f'][-1]:.6f}")
    print(f"Momentum converges faster: {hist_mom['f'][-1] < hist_vanilla['f'][-1]}")
else:
    print("TODO: Implement the momentum update in the loop above!")
    print("Expected: Momentum should converge significantly faster than vanilla GD")
    print(f"Vanilla GD final loss: {hist_vanilla['f'][-1]:.6f}")

### Exercise 2: Cross-Validation for Model Selection

Use cross-validation to find the best polynomial degree for a noisy dataset. This connects the bias-variance tradeoff to practical model selection.

In [ ]:
# EXERCISE 2: Find the best polynomial degree using cross-validation
np.random.seed(42)

# Generate data
n_cv = 100
x_cv = np.random.uniform(0, 2*np.pi, n_cv)
y_cv = np.sin(x_cv) + 0.3 * np.cos(3*x_cv) + 0.4 * np.random.randn(n_cv)

def evaluate_polynomial_degree(x, y, degree, cv_folds=5):
    """
    Evaluate a polynomial regression model using cross-validation.

    Args:
        x: Input features (1D array)
        y: Target values
        degree: Polynomial degree to evaluate
        cv_folds: Number of cross-validation folds

    Returns:
        mean_cv_score: Mean cross-validation score (negative MSE)
    """
    # TODO: Implement this!
    # Hint:
    # 1. Create polynomial features: PolynomialFeatures(degree)
    # 2. Transform x: poly.fit_transform(x.reshape(-1, 1))
    # 3. Use cross_val_score with LinearRegression and scoring='neg_mean_squared_error'
    # 4. Return the mean score

    pass  # Replace with your implementation

# Test all degrees from 1 to 15
degrees_cv = range(1, 16)
cv_scores = []

for d in degrees_cv:
    score = evaluate_polynomial_degree(x_cv, y_cv, d)
    if score is not None:
        cv_scores.append(-score)  # Negate because sklearn returns negative MSE
    else:
        cv_scores.append(None)

if cv_scores[0] is not None:
    best_degree = degrees_cv[np.argmin(cv_scores)]
    print(f"Best polynomial degree by CV: {best_degree}")
    print(f"CV MSE scores: {[f'{s:.4f}' for s in cv_scores]}")

    # Verify
    expected_best = 4  # Approximate expected best (sin + cos terms)
    print(f"\nExpected best degree: around {expected_best}")
    print(f"Your best degree: {best_degree}")
    print(f"Reasonable: {1 <= best_degree <= 7}")
else:
    print("TODO: Implement evaluate_polynomial_degree above!")
    print("Expected: Best degree should be around 3-5 (matching the true function complexity)")

### Exercise 3: Compare L1, L2, and Elastic Net Regularization

Fit models with different regularization strategies on a dataset with many irrelevant features. Analyze which method best recovers the true sparse structure.

In [ ]:
# EXERCISE 3: Regularization comparison
np.random.seed(42)

# Create dataset with sparse true weights
n_samples, n_features_ex = 80, 30
X_ex = np.random.randn(n_samples, n_features_ex)
w_true_ex = np.zeros(n_features_ex)
w_true_ex[:8] = [4, -3, 2, -1.5, 1, -0.5, 3, -2]  # Only 8 features matter
y_ex = X_ex @ w_true_ex + 0.5 * np.random.randn(n_samples)

def compare_regularizers(X, y, w_true, alpha_l1=0.5, alpha_l2=1.0, l1_ratio=0.5):
    """
    Compare L1, L2, and Elastic Net regularization.

    Args:
        X: Feature matrix
        y: Target vector
        w_true: True weight vector (for comparison)
        alpha_l1: Regularization strength for Lasso
        alpha_l2: Regularization strength for Ridge
        l1_ratio: L1 ratio for Elastic Net

    Returns:
        dict with 'lasso', 'ridge', 'elastic_net' entries, each containing:
            'coef': fitted coefficients
            'n_nonzero': number of non-zero coefficients (|w| > 0.01)
            'mse': mean squared error of coefficient recovery
    """
    results = {}

    # TODO: Implement this!
    # Hint:
    # 1. Fit Lasso(alpha=alpha_l1), Ridge(alpha=alpha_l2),
    #    and ElasticNet (from sklearn.linear_model import ElasticNet)
    # 2. For each, compute:
    #    - coef: model.coef_
    #    - n_nonzero: np.sum(np.abs(model.coef_) > 0.01)
    #    - mse: np.mean((model.coef_ - w_true)**2)

    pass  # Replace with your implementation

    return results

# Test
results = compare_regularizers(X_ex, y_ex, w_true_ex)

if results:
    print("Regularization Comparison:")
    print(f"{'Method':<15} {'Non-zero':>10} {'Coef MSE':>10} {'True non-zero':>15}")
    print("-" * 55)
    true_nonzero = np.sum(np.abs(w_true_ex) > 0.01)
    for name, res in results.items():
        print(f"{name:<15} {res['n_nonzero']:>10} {res['mse']:>10.4f} {true_nonzero:>15}")

    # Verify
    print(f"\nExpected: Lasso should have fewest non-zero weights (closest to {true_nonzero})")
    print(f"Expected: Ridge should have ALL weights non-zero")
    print(f"Expected: Elastic Net should be in between")
else:
    print("TODO: Implement compare_regularizers above!")
    print(f"True non-zero weights: {np.sum(np.abs(w_true_ex) > 0.01)}")
    print("Expected: L1 finds sparsity, L2 shrinks all, Elastic Net balances both")

### Exercise 4 (Bonus): Learning Rate Schedules

Implement and compare three learning rate schedules for SGD:
1. **Constant**: $\eta_t = \eta_0$
2. **Step decay**: $\eta_t = \eta_0 \cdot 0.5^{\lfloor t/50 \rfloor}$
3. **Cosine annealing**: $\eta_t = \eta_0 \cdot \frac{1}{2}\left(1 + \cos\left(\frac{\pi t}{T}\right)\right)$

In [ ]:
# EXERCISE 4 (BONUS): Learning rate schedules
np.random.seed(42)

def lr_constant(t, lr0=0.01):
    return lr0

def lr_step_decay(t, lr0=0.01, drop_rate=0.5, drop_every=50):
    """Step decay: halve the learning rate every drop_every steps."""
    # TODO: Implement this
    # Hint: return lr0 * drop_rate ** (t // drop_every)
    pass

def lr_cosine(t, lr0=0.01, T=200):
    """Cosine annealing schedule."""
    # TODO: Implement this
    # Hint: return lr0 * 0.5 * (1 + np.cos(np.pi * t / T))
    pass

# Compare on the least-squares problem from Section 3
schedules = {
    'Constant': lr_constant,
    'Step Decay': lr_step_decay,
    'Cosine Annealing': lr_cosine
}

n_steps_sched = 200

for name, schedule in schedules.items():
    if schedule is None or schedule(0) is None:
        print(f"TODO: Implement {name} schedule")
        continue
    np.random.seed(42)
    hist = sgd(stochastic_grad, w0, schedule, n_steps_sched, N, batch_size=16, f=full_loss)
    print(f"{name}: final loss = {hist['f'][-1]:.6f}")

print("\nExpected: Cosine annealing or step decay should outperform constant LR")

---
## Summary

### Key Concepts

- **Gradient descent convergence** depends on convexity and the condition number. Strongly convex functions converge exponentially; convex functions converge as $O(1/t)$.
- **Step size selection** is critical: too large causes divergence, too small wastes compute. The optimal step size is $\eta = 1/L$ where $L$ is the Lipschitz constant.
- **SGD** trades exact gradients for computational efficiency, and the noise provides implicit regularization that often improves generalization.
- **Bias-variance tradeoff** decomposes test error into underfitting (bias) and overfitting (variance). Model complexity must be matched to the problem.
- **VC dimension** measures model capacity — the largest dataset a model class can perfectly memorize for any labeling.
- **PAC learning** provides sample complexity bounds: how much data you need for a given accuracy and confidence level.
- **L1 regularization** produces sparse solutions (feature selection) while **L2 regularization** produces small weights. This follows from the geometry of their constraint regions.
- **Overparameterized networks** generalize better than classical theory predicts, due to implicit regularization, the double descent phenomenon, and the lottery ticket hypothesis.

### Connection to Machine Learning

| Concept | ML Application | Why It Matters |
|---------|---------------|----------------|
| Convergence rates | Learning rate tuning | Knowing the theory prevents trial-and-error |
| Condition number | Feature normalization, BatchNorm | Explains why preprocessing helps |
| SGD noise | Generalization, escaping local minima | Why SGD often beats full-batch GD |
| Bias-variance | Model selection, hyperparameter tuning | The fundamental tradeoff in all of ML |
| VC dimension | Choosing model architecture | Capacity must match data complexity |
| PAC learning | Dataset size requirements | How much data you actually need |
| L1/L2 regularization | Dropout, weight decay, pruning | Most common tools against overfitting |
| Double descent | Why large models work | Challenges classical "simpler is better" wisdom |
| Lottery tickets | Network pruning, efficient inference | 90%+ of weights may be unnecessary |

### Checklist

- [ ] I can explain why strongly convex functions converge faster than convex ones
- [ ] I can choose an appropriate learning rate given a function's properties
- [ ] I understand why SGD's noise is a feature, not a bug
- [ ] I can decompose test error into bias, variance, and irreducible noise
- [ ] I can explain what VC dimension measures and why it matters
- [ ] I can use PAC bounds to estimate required dataset sizes
- [ ] I understand geometrically why L1 gives sparsity (diamond vs sphere)
- [ ] I can explain the double descent curve and why overparameterization helps
- [ ] I know what the lottery ticket hypothesis claims and its implications

---
## Next Steps

In **Part 2.1: Python OOP for ML** (Notebook 07), we shift from mathematical foundations to software foundations. You'll learn the object-oriented programming patterns that underpin every ML framework — from PyTorch's `nn.Module` to scikit-learn's estimator API. The optimization theory from this notebook will come alive when you implement gradient descent as a proper `Optimizer` class.

**Key connections to look forward to:**
- The `Optimizer` class pattern (SGD, Adam, etc.) encapsulates everything from Sections 1-3
- Regularization shows up as `weight_decay` parameters in optimizers
- Model selection (bias-variance) drives hyperparameter search pipelines
- Everything in this notebook becomes practical when you build training loops in Part 3